# This is Main file, containing all the work flow.

## Python & SQL & Machine Learning Project
### Use Decision Tree regression:
	1. Select 10 stocks as independent variables to predict stocks return for the left 2990 stocks
	2. Training sample should be at least 300 daily returns, and test sample at least 200 days.
	3. Determine which stock has the smallest out-of-sample RMSE
    4. Data Retrieval
     Get data from data source from yahoo
     Save data to the database
     Output result to DB




### 1. Import Packages

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
import math as mh
import fix_yahoo_finance as yf
import datetime
import pyodbc

### 2. Read Russel 3000 tickers from Local file

In [ ]:
# read tickers from csv
symbol = pd.read_csv("symbol.csv")
# reset column names
symbol.columns = ['Company','Ticker']
# get ticker list 
ticker = list(symbol['Ticker'])

### 3. Retrive data from yahoo finance 

In [ ]:
#retrive data for last two years from fix_yahoo_finance
stocks = ticker
start = datetime.date(2016,12,10)
end = datetime.date(2018,12,10)
data = yf.download(stocks, start=start, end=end)
# get close price from dataframe 'data'
Close_data = data.Close
# drop data with NA value for the last two years
Close = Close_data.dropna(axis = 'columns')

### 4. Export user-input and original price data from python to database  

In [ ]:
from Model_Output import User_Output, Price_Output
# set user-input data
run_id = 1
train_period = 300
test_period = 200
start_date = '2016-12-10'
end_date = '2018-12-10'

# export user-input data
User_Output(run_id, train_period, test_period, start_date, end_date)

In [ ]:
# export price data 
Price_Output(Close)

### 5. Import user-input data and price data from database to python 

In [ ]:
from Model_Input import Data_Input

# set the test run_id as 1
run_test = 1

# Notes: Each run_id is unique. It contains all the unique information for this run.
# This means we only need one parameter to get data we need.

# Get one dataframe contaning price data from DB
price_db = Data_Input(run_test)['df_price']
# Get one dataframe contaning user-input data from DB
# Data includes Start_Date, End_Date,Train_Period,Test_Period
user_db = Data_Input(run_test)['df_dates']

### 6. Prepare data for regression training 

In [ ]:
# Convert DB dataframe to new dataframe where all columns' names are tickers
# get all stock tickers
price_db_tickers = list(set(price_db['Ticker']))
# get all dates(use set to get unique value)
price_db_dates = list(set(price_db['AsofDate']))
# get the new Price dictionary where keys are tickers and values are their prices
Db_data = {}
for i in range(len(price_db_tickers)):
    Db_data[price_db_tickers[i]] = []
    for j in range(len(price_db_dates)):
        temp_p = price_db.loc[(price_db['Ticker'] == price_db_tickers[i]) & 
                              (price_db['AsofDate'] == price_db_dates[j])]
        Db_data[price_db_tickers[i]].append(temp_p['Price'])
# transform dictionary to dataframe
Db_data = pd.DataFrame(Db_data)        

In [ ]:
# Convert price data into log return data
Log_ret =  np.log(Db_data / Db_data.shift(1))
# drop na values in rows
Ori_data = Log_ret.dropna(axis = 'rows')
# Get all the ticker names
col_names = list(Ori_data.columns)
# Get the number of dates
n_row = len(Ori_data)
# Get all the dates and convert into string types
Ori_date = list(Ori_data.index) 
for i in range(n_row):
    Ori_date[i] = str(Ori_date[i].date())

In [ ]:
# get the selected prdictors from csv file
f_symbol = pd.read_csv("SELECT.csv")
features = list(f_symbol['TICKER'])
# get the left dependent variables
dependents = [i for i in col_names if i not in features]
# get the number of features and dependent variables
n_predicor = len(dependents)

### 7. Use Desicion Tree Regression model to train the data

In [8]:
# define a function to calculate the RMSE for returns
def calc_mse(l1,l2):
    length = len(l1)
    mse = 0
    for i in range(length):
        mse = mse + mh.pow((l1[i]-l2[i]),2)
    return(mh.sqrt(mse))

In [ ]:
# train the model
# set the predicted return as a dictionary(further into a dataframe)
Ret_predict = {}
# set the predicted return MSE as a list
Ret_MSE = []

In [ ]:
# get the number of training period
t_p = user_db['Train_Period'][0]
s_p = user_db['Test_Period'][0]

In [ ]:
# use last day return to predict next day return using pre-selected stocks
# seperate training set and testing set for independent variables
x_train = np.array(Ori_data[features][:t_p])
x_train = np.reshape(x_train,(t_p,len(features)))
x_test = np.array(Ori_data[features][t_p+1: t_p+s_p])
# using selected stocks to train each stock left
for i in range(n_predicor):
    # seperate training set and testing set for each dependent variables
    y_train = np.array(list(Ori_data[dependents[i]])[1:t_p + 1])
    y_test = np.array(list(Ori_data[dependents[i]])[t_p + 2 : t_p+s_p+1])
    # run regression tree model
    regr = DecisionTreeRegressor(max_depth=10)
    # model fitting
    regr.fit(x_train, y_train)
    # model prediction
    y_pre = regr.predict(x_test)
    # get dictionary for return prediction
    Ret_predict[dependents[i]] = y_pre 
    # get the list for MSE value 
    Ret_MSE.append(calc_mse(y_test,y_pre))
# get the dataframe for Return prediction    
Ret_predict = pd.DataFrame(Ret_predict,index = Ori_data.index[t_p + 2 : t_p+s_p+1]

### 8. Export return prediction and mse value from python to database  

In [ ]:
# Model result export
from Model_Output import Result_Output
Result_Output(run_id,features,Ret_predict,Ret_MSE)